In [1]:
import numpy as np

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from queue import PriorityQueue


In [2]:
with open("input_day_19.txt") as file:
    
    text = file.read()



In [3]:
test = False
if test:

    text = """
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
"""
    
text = text.strip()

In [4]:
lines = text.split("\n")

words = [l.split() for l in lines]

ore_robot_cost = [int(w[6]) for w in words] # ore units
clay_robot_cost = [int(w[12]) for w in words] # ore units
obsidian_robot_cost = [(int(w[18]), int(w[21])) for w in words] # ore and clay units
geode_robot_cost = [(int(w[27]), int(w[30])) for w in words] # ore and obsidian units


In [5]:
ore_robot_cost
clay_robot_cost
obsidian_robot_cost
geode_robot_cost

[(4, 9),
 (2, 10),
 (4, 13),
 (4, 8),
 (2, 10),
 (3, 19),
 (4, 14),
 (2, 14),
 (2, 16),
 (2, 18),
 (4, 10),
 (3, 15),
 (3, 7),
 (4, 17),
 (3, 20),
 (4, 20),
 (2, 11),
 (3, 14),
 (2, 7),
 (3, 10),
 (2, 19),
 (2, 19),
 (2, 15),
 (3, 11),
 (2, 12),
 (3, 15),
 (3, 13),
 (2, 13),
 (4, 9),
 (3, 16)]

In [6]:
# currently gets the number assuming no more robots are build
def estimated_total_geodes_old(state):
    robot_counts, material_counts, minutes_left = state
    
    return material_counts[3] + robot_counts[3] * minutes_left

# adds a factor for preferring more robots
def estimated_total_geodes(state):
    
    robot_counts, material_counts, minutes_left = state
    
    geodes_if_no_more_building = material_counts[3] + robot_counts[3] * minutes_left
    estimated_robot_values = robot_counts[0] + 10 * robot_counts[1] + 100 * robot_counts[2] + 1000 * robot_counts[3]
    estimated_material_values = material_counts[0] + 10 * material_counts[1] + 100 * material_counts[2] # obsidian most valuable
    
    return geodes_if_no_more_building + 0.1 * estimated_robot_values + 0.1 * estimated_material_values


In [7]:
actual_states = [
    ((1, 0, 0, 0), (0, 0, 0, 0), 24), # after minute 0
    ((1, 0, 0, 0), (1, 0, 0, 0), 23), # after minute 1
    ((1, 0, 0, 0), (2, 0, 0, 0), 22), # 2
    ((1, 1, 0, 0), (1, 0, 0, 0), 21), # 3
    ((1, 1, 0, 0), (2, 1, 0, 0), 20), # 4
    ((1, 2, 0, 0), (1, 2, 0, 0), 19), # 5
    ((1, 2, 0, 0), (2, 4, 0, 0), 18), # 6
    ((1, 3, 0, 0), (1, 6, 0, 0), 17), # 7
    ((1, 3, 0, 0), (2, 9, 0, 0), 16), # 8
    ((1, 3, 0, 0), (3, 12, 0, 0), 15), # 9
    ((1, 3, 0, 0), (4, 15, 0, 0), 14), # 10
    
    ((1, 3, 1, 0), (2, 4, 0, 0), 13), # 11
    ((1, 3, 1, 0), (2, 4, 0, 0), 12), # 12
    ((1, 3, 1, 0), (2, 4, 0, 0), 11), # 13
    ((1, 3, 1, 0), (2, 4, 0, 0), 10), # 14
    ((1, 3, 1, 0), (2, 4, 0, 0), 9), # 15
    ((1, 3, 1, 0), (2, 4, 0, 0), 8), # 16
    ((1, 3, 1, 0), (2, 4, 0, 0), 7), # 17
    ((1, 3, 1, 0), (2, 4, 0, 0), 6), # 18
    ((1, 3, 1, 0), (2, 4, 0, 0), 5), # 19
    ((1, 3, 1, 0), (2, 4, 0, 0), 4), # 20
    ((1, 3, 1, 0), (2, 4, 0, 0), 3), # 21
    ((1, 3, 1, 0), (2, 4, 0, 0), 2), # 22
    ((1, 3, 1, 0), (2, 4, 0, 0), 1), # 23
    ((1, 3, 1, 0), (2, 4, 0, 0), 0), # 24
]
#debugging, built obsidian robot ((1, 3, 1, 0), (2, 4, 0, 0), 13)

In [8]:
do_part_1 = False
if do_part_1:

    num_geodes = []

    for blueprint in range(1, len(lines) + 1):

        max_geode_count = 0

        print(f"\nblueprint {blueprint}:")

        q = PriorityQueue()

        # robot counts, material counts, minutes left
        current_state = ((1, 0, 0, 0), (0, 0, 0, 0), 24) 

        q.put((-estimated_total_geodes(current_state), current_state))

        prev_considered_state_hashes = set()

        num_states_considered = 0

        while not q.empty():

            estimated_geoges, current_state = q.get()

            if num_states_considered % 5000000 == 0:
                print(f"\n    states considered: {num_states_considered}\n    queue size: {q.qsize()}\n    max_geode_count: {max_geode_count}\n    {current_state}")

            num_states_considered += 1


            current_hash = hash(current_state)
            if current_hash in prev_considered_state_hashes:
                continue
            else:
                prev_considered_state_hashes.add(current_hash)

            robot_counts, material_counts, minutes_left = current_state
            ore_robot_count, clay_robot_count, obsidian_robot_count, geode_robot_count = robot_counts
            ore_count, clay_count, obsidian_count, geode_count = material_counts


            if geode_count > max_geode_count:
                max_geode_count = geode_count
                print("    new best:", geode_count)

            if minutes_left == 0:
                continue


            max_possible_geode_count = geode_count + geode_robot_count * minutes_left
            for m in range(minutes_left): # 1 min left is zero, 2 mins left is 0 + 1, 3 mins left is 0 + 1 + 2
                max_possible_geode_count += m

            if max_possible_geode_count <= max_geode_count:
                continue


            next_material_counts = tuple([material_counts[i] + robot_counts[i] for i in range(4)])
            next_ore_count, next_clay_count, next_obsidian_count, next_geode_count = next_material_counts

            new_robot_options = (0, 1, 2, 3)


            #if current_state in actual_states and minutes_left == 14:
            #    print("found it", current_state)

            #if current_state in actual_states and minutes_left == 14:
            #    print("debugging, options", new_robot_options)


            geode_ore_cost = geode_robot_cost[blueprint-1][0]
            geode_obsidian_cost = geode_robot_cost[blueprint-1][1]
            if geode_ore_cost <= ore_count and geode_obsidian_cost <= obsidian_count: # must build a geode robot if you can
                new_robot_options = [4]

            for new_robot_type in new_robot_options:

                if new_robot_type == 0: # don't build a robot                
                    next_state = (robot_counts, next_material_counts, minutes_left - 1)
                    q.put((-estimated_total_geodes(next_state), next_state))

                elif new_robot_type == 1: # build an ore robot
                    ore_cost = ore_robot_cost[blueprint-1]
                    if ore_cost > ore_count:
                        continue
                    else:
                        next_material_counts = (next_ore_count - ore_cost, next_clay_count, next_obsidian_count, next_geode_count)
                        next_robot_counts = (ore_robot_count + 1, clay_robot_count, obsidian_robot_count, geode_robot_count)
                        next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                        q.put((-estimated_total_geodes(next_state), next_state))

                elif new_robot_type == 2: # build a clay robot
                    ore_cost = clay_robot_cost[blueprint-1]
                    if ore_cost > ore_count:
                        continue
                    else:
                        next_material_counts = (next_ore_count - ore_cost, next_clay_count, next_obsidian_count, next_geode_count)
                        next_robot_counts = (ore_robot_count, clay_robot_count + 1, obsidian_robot_count, geode_robot_count)
                        next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                        q.put((-estimated_total_geodes(next_state), next_state))

                elif new_robot_type == 3: # build an obsidian robot

                    ore_cost = obsidian_robot_cost[blueprint-1][0]
                    clay_cost = obsidian_robot_cost[blueprint-1][1]
                    if ore_cost > ore_count or clay_cost > clay_count:
                        continue
                    else:
                        next_material_counts = (next_ore_count - ore_cost, next_clay_count - clay_cost, next_obsidian_count, next_geode_count)
                        next_robot_counts = (ore_robot_count, clay_robot_count, obsidian_robot_count + 1, geode_robot_count)
                        next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                        q.put((-estimated_total_geodes(next_state), next_state))

                elif new_robot_type == 4: # build a geode robot
                    # already checked costs
                    next_material_counts = (next_ore_count - geode_ore_cost, next_clay_count, next_obsidian_count - geode_obsidian_cost, next_geode_count)
                    next_robot_counts = (ore_robot_count, clay_robot_count, obsidian_robot_count, geode_robot_count + 1)
                    next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                    q.put((-estimated_total_geodes(next_state), next_state))



        print(f"all states ({num_states_considered}), exhausted, max geodes = {max_geode_count}")
        num_geodes.append(max_geode_count)




In [9]:
if do_part_1:
    print("geodes:", num_geodes)
    print("quality levels:", [num_geodes[i] * (i+1) for i in range(len(lines))])
    print("total quality level:", sum([num_geodes[i] * (i+1) for i in range(len(lines))]))


In [10]:
num_geodes = []

for blueprint in range(1, min(len(lines), 3) + 1):
    
    max_geode_count = 0
    
    print(f"\nblueprint {blueprint}:")
    
    q = PriorityQueue()
    
    # robot counts, material counts, minutes left
    current_state = ((1, 0, 0, 0), (0, 0, 0, 0), 32) 
    
    q.put((-estimated_total_geodes(current_state), current_state))

    prev_considered_state_hashes = set()
    
    num_states_considered = 0

    while not q.empty():
        
        estimated_geoges, current_state = q.get()
                
        if num_states_considered % 5000000 == 0:
            print(f"\n    states considered: {num_states_considered}\n    queue size: {q.qsize()}\n    max_geode_count: {max_geode_count}\n    {current_state}")
            
        num_states_considered += 1

        
        current_hash = hash(current_state)
        if current_hash in prev_considered_state_hashes:
            continue
        else:
            prev_considered_state_hashes.add(current_hash)
            
        robot_counts, material_counts, minutes_left = current_state
        ore_robot_count, clay_robot_count, obsidian_robot_count, geode_robot_count = robot_counts
        ore_count, clay_count, obsidian_count, geode_count = material_counts
        
        
        if geode_count > max_geode_count:
            max_geode_count = geode_count
            print("    new best:", geode_count)
        
        if minutes_left == 0:
            continue
            
            
        max_possible_geode_count = geode_count + geode_robot_count * minutes_left
        for m in range(minutes_left): # 1 min left is zero, 2 mins left is 0 + 1, 3 mins left is 0 + 1 + 2
            max_possible_geode_count += m
            
        if max_possible_geode_count <= max_geode_count:
            continue
        
        
        next_material_counts = tuple([material_counts[i] + robot_counts[i] for i in range(4)])
        next_ore_count, next_clay_count, next_obsidian_count, next_geode_count = next_material_counts
            
        new_robot_options = (0, 1, 2, 3)
        
        
        #if current_state in actual_states and minutes_left == 14:
        #    print("found it", current_state)
            
        #if current_state in actual_states and minutes_left == 14:
        #    print("debugging, options", new_robot_options)
        
        
        geode_ore_cost = geode_robot_cost[blueprint-1][0]
        geode_obsidian_cost = geode_robot_cost[blueprint-1][1]
        if geode_ore_cost <= ore_count and geode_obsidian_cost <= obsidian_count: # must build a geode robot if you can
            new_robot_options = [4]
                    
        for new_robot_type in new_robot_options:

            if new_robot_type == 0: # don't build a robot                
                next_state = (robot_counts, next_material_counts, minutes_left - 1)
                q.put((-estimated_total_geodes(next_state), next_state))

            elif new_robot_type == 1: # build an ore robot
                ore_cost = ore_robot_cost[blueprint-1]
                if ore_cost > ore_count:
                    continue
                else:
                    next_material_counts = (next_ore_count - ore_cost, next_clay_count, next_obsidian_count, next_geode_count)
                    next_robot_counts = (ore_robot_count + 1, clay_robot_count, obsidian_robot_count, geode_robot_count)
                    next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                    q.put((-estimated_total_geodes(next_state), next_state))
                    
            elif new_robot_type == 2: # build a clay robot
                ore_cost = clay_robot_cost[blueprint-1]
                if ore_cost > ore_count:
                    continue
                else:
                    next_material_counts = (next_ore_count - ore_cost, next_clay_count, next_obsidian_count, next_geode_count)
                    next_robot_counts = (ore_robot_count, clay_robot_count + 1, obsidian_robot_count, geode_robot_count)
                    next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                    q.put((-estimated_total_geodes(next_state), next_state))
                    
            elif new_robot_type == 3: # build an obsidian robot
                
                ore_cost = obsidian_robot_cost[blueprint-1][0]
                clay_cost = obsidian_robot_cost[blueprint-1][1]
                if ore_cost > ore_count or clay_cost > clay_count:
                    continue
                else:
                    next_material_counts = (next_ore_count - ore_cost, next_clay_count - clay_cost, next_obsidian_count, next_geode_count)
                    next_robot_counts = (ore_robot_count, clay_robot_count, obsidian_robot_count + 1, geode_robot_count)
                    next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                    q.put((-estimated_total_geodes(next_state), next_state))
                    
            elif new_robot_type == 4: # build a geode robot
                # already checked costs
                next_material_counts = (next_ore_count - geode_ore_cost, next_clay_count, next_obsidian_count - geode_obsidian_cost, next_geode_count)
                next_robot_counts = (ore_robot_count, clay_robot_count, obsidian_robot_count, geode_robot_count + 1)
                next_state = (next_robot_counts, next_material_counts, minutes_left - 1)
                q.put((-estimated_total_geodes(next_state), next_state))
        
        
        
    print(f"all states ({num_states_considered}), exhausted, max geodes = {max_geode_count}")
    num_geodes.append(max_geode_count)





blueprint 1:

    states considered: 0
    queue size: 0
    max_geode_count: 0
    ((1, 0, 0, 0), (0, 0, 0, 0), 32)
    new best: 1
    new best: 2
    new best: 3
    new best: 4
    new best: 5
    new best: 6
    new best: 7
    new best: 8
    new best: 9

    states considered: 5000000
    queue size: 16
    max_geode_count: 9
    ((4, 5, 2, 0), (10, 8, 3, 0), 8)
    new best: 10

    states considered: 10000000
    queue size: 20
    max_geode_count: 10
    ((5, 3, 0, 0), (22, 12, 0, 0), 8)
    new best: 11

    states considered: 15000000
    queue size: 29
    max_geode_count: 11
    ((6, 7, 2, 0), (18, 50, 1, 0), 5)
    new best: 12
    new best: 13
    new best: 14

    states considered: 20000000
    queue size: 27
    max_geode_count: 14
    ((8, 6, 2, 1), (44, 50, 4, 1), 4)
    new best: 15
    new best: 16
all states (23717580), exhausted, max geodes = 16

blueprint 2:

    states considered: 0
    queue size: 0
    max_geode_count: 0
    ((1, 0, 0, 0), (0, 0, 0, 0), 32

In [11]:
print(num_geodes)
print(num_geodes[0] * num_geodes[1] * num_geodes[2])

[16, 54, 29]
25056
